# Using Versioning in S3 Bucket

This notebook is a detailed, interactive tutorial for learning.

Reference:

- [Using Versioning in S3 Bucket](https://docs.aws.amazon.com/AmazonS3/latest/userguide/Versioning.html)

## Prepare Your Playground

First, we need to prepare our development environment for a better learning experience.

- an AWS CLI Profile with access to S3, it should have S3 full access and STS get-caller-identity permission.
- a bucket with versioning turned on (we will create it soon)
- Installed the following python library:
    - [boto_session_manager](https://pypi.org/project/boto_session_manager/): boto3 session management made easy
    - [s3pathlib](https://pypi.org/project/s3pathlib/): s3 manipulation made easy
    - [rich](https://pypi.org/project/s3pathlib/): for pretty print

In [1]:
# Enter your AWS Profile here
aws_profile = "awshsh_app_dev_us_east_1"

In [13]:
from rich import print as rprint

def rprint_response(res: dict):
    if "ResponseMetadata" in res:
        res.pop("ResponseMetadata")
    rprint(res)

In [18]:
from boto_session_manager import BotoSesManager
from s3pathlib import S3Path, context

bsm = BotoSesManager(profile_name=aws_profile)
context.attach_boto_session(bsm.boto_ses)

bucket = f"{bsm.aws_account_id}-{bsm.aws_region}-learn-s3-versioning"

# Create the bucket and turn on versioning
def is_bucket_exists() -> bool:
    try:
        bsm.s3_client.head_bucket(Bucket=bucket)
        return True
    except bsm.s3_client.exceptions.ClientError as e:
        return False

if is_bucket_exists() is False:
    kwargs = dict(Bucket=bucket)
    if bsm.aws_region != "us-east-1":
        kwargs["CreateBucketConfiguration"] = dict(LocationConstraint=bsm.aws_region)
    bsm.s3_client.create_bucket(**kwargs)

response = bsm.s3_client.get_bucket_versioning(
    Bucket=bucket,
)
if "Status" in response: # versioning is already enabled or suspended
    pass
else: # versioning is not enabled
    bsm.s3_client.put_bucket_versioning(
        Bucket=bucket,
        VersioningConfiguration=dict(
            Status="Enabled",
        )
    )

# verify if bucket versioning is enabled
response = bsm.s3_client.get_bucket_versioning(
    Bucket=bucket,
)
rprint_response(response)
print(f"preview S3 bucket: {S3Path(bucket).console_url}")

{'Status': 'Enabled'}

preview S3 bucket: https://console.aws.amazon.com/s3/buckets/807388292768-us-east-1-learn-s3-versioning?tab=objects


## Put and Get

[EN]

First, we create a new object in a bucket with versioning turned on. It is also the first version of this object

[CN]

In [71]:
s3path = S3Path(bucket, "test.txt")
res = bsm.s3_client.put_object(
    Bucket=bucket,
    Key=key,
    Body="v1",
)
rprint(res)
v1 = res["VersionId"]
print(f"The version id = {v1}")

{
    'ResponseMetadata': {
        'RequestId': 'MGA3BC1YTC6CQKFY',
        'HostId': 'ZheaTmKtemyJU1yhTdVrlynmd98ITJefiH+BfjtqnBezkj5vrDsJ2E9mynv7m6wPTR66ls4Dbgk=',
        'HTTPStatusCode': 200,
        'HTTPHeaders': {
            'x-amz-id-2': 'ZheaTmKtemyJU1yhTdVrlynmd98ITJefiH+BfjtqnBezkj5vrDsJ2E9mynv7m6wPTR66ls4Dbgk=',
            'x-amz-request-id': 'MGA3BC1YTC6CQKFY',
            'date': 'Sat, 15 Apr 2023 21:35:37 GMT',
            'x-amz-version-id': 'BLoyX9X9xLmHSx8gLgNzWUKI7kwwkuNz',
            'x-amz-server-side-encryption': 'AES256',
            'etag': '"6654c734ccab8f440ff0825eb443dc7f"',
            'server': 'AmazonS3',
            'content-length': '0'
        },
        'RetryAttempts': 0
    },
    'ETag': '"6654c734ccab8f440ff0825eb443dc7f"',
    'ServerSideEncryption': 'AES256',
    'VersionId': 'BLoyX9X9xLmHSx8gLgNzWUKI7kwwkuNz'
}

The version id = BLoyX9X9xLmHSx8gLgNzWUKI7kwwkuNz


Then we can immediately get the object. By default, the latest version is returned.

In [72]:
res = bsm.s3_client.get_object(Bucket=bucket, Key=key)
rprint(res)
print("Content = {}".format(res["Body"].read().decode("utf-8")))
print("The version id = {}".format(res["VersionId"]))

{
    'ResponseMetadata': {
        'RequestId': 'N04YGWXDG506FP38',
        'HostId': 'YF+dDH27krg5vdAOg0iOttDImX8tI71Y0aKuH+JtIGKVz0+9BOEh1iwwShJwg7wx4JHI0gnzRP8=',
        'HTTPStatusCode': 200,
        'HTTPHeaders': {
            'x-amz-id-2': 'YF+dDH27krg5vdAOg0iOttDImX8tI71Y0aKuH+JtIGKVz0+9BOEh1iwwShJwg7wx4JHI0gnzRP8=',
            'x-amz-request-id': 'N04YGWXDG506FP38',
            'date': 'Sat, 15 Apr 2023 21:35:38 GMT',
            'last-modified': 'Sat, 15 Apr 2023 21:35:37 GMT',
            'etag': '"6654c734ccab8f440ff0825eb443dc7f"',
            'x-amz-server-side-encryption': 'AES256',
            'x-amz-version-id': 'BLoyX9X9xLmHSx8gLgNzWUKI7kwwkuNz',
            'accept-ranges': 'bytes',
            'content-type': 'binary/octet-stream',
            'server': 'AmazonS3',
            'content-length': '2'
        },
        'RetryAttempts': 0
    },
    'AcceptRanges': 'bytes',
    'LastModified': datetime.datetime(2023, 4, 15, 21, 35, 37, tzinfo=tzutc()),
    'ContentLength': 2,
    'ETag': '"6654c734ccab8f440ff0825eb443dc7f"',
    'VersionId': 'BLoyX9X9xLmHSx8gLgNzWUKI7kwwkuNz',
    'ContentType': 'binary/octet-stream',
    'ServerSideEncryption': 'AES256',
    'Metadata': {},
    'Body': <botocore.response.StreamingBody object at 0x1080ab100>
}

Content = v1
The version id = BLoyX9X9xLmHSx8gLgNzWUKI7kwwkuNz


Then we put a new content to this object, which creates a new version of the object. Note that you cannot overwrite an existing version, because the versioning system is designed to ensure immutability.

In [73]:
res = bsm.s3_client.put_object(
    Bucket=bucket,
    Key=key,
    Body="v2",
)
v2 = res["VersionId"]
print(f"The version id = {v2}")

The version id = 36dbatEjGuecZop4MARBZMwfORr4k9Hc


We get the object again, by default, it get the latest version, and we can see that the version id is different from the previous one.

In [74]:
res = bsm.s3_client.get_object(Bucket=bucket, Key=key)

content = res["Body"].read().decode("utf-8")
assert content == "v2"
print(f"Content = {content}")

v = res["VersionId"]
assert v == v2
print(f"The version id = {v}")

Content = v2
The version id = 36dbatEjGuecZop4MARBZMwfORr4k9Hc


We can explicitly get a historical version using version id

In [75]:
res = bsm.s3_client.get_object(Bucket=bucket, Key=key, VersionId=v1)

content = res["Body"].read().decode("utf-8")
assert content == "v1"
print(f"Content = {content}")

v = res["VersionId"]
assert v == v1
print(f"The version id = {v}")

Content = v1
The version id = BLoyX9X9xLmHSx8gLgNzWUKI7kwwkuNz


We can also list all the historical version of an object. It will return in order of last modified time, from the latest to the oldest.

In [76]:
res = bsm.s3_client.list_object_versions(Bucket=bucket, Prefix=key)
rprint(res)

n_versions = len(res["Versions"])
print(f"Number of versions = {n_versions}")

{
    'ResponseMetadata': {
        'RequestId': 'FNKD6K299KHXVXCA',
        'HostId': '++HHrB2WzsAxEbk93iRaxU8gvAw8MKJ5BKf2/TZ7nQFZJis7EUwPF4yjYydY6QrqxAz60E4yiz0=',
        'HTTPStatusCode': 200,
        'HTTPHeaders': {
            'x-amz-id-2': '++HHrB2WzsAxEbk93iRaxU8gvAw8MKJ5BKf2/TZ7nQFZJis7EUwPF4yjYydY6QrqxAz60E4yiz0=',
            'x-amz-request-id': 'FNKD6K299KHXVXCA',
            'date': 'Sat, 15 Apr 2023 21:35:39 GMT',
            'content-type': 'application/xml',
            'transfer-encoding': 'chunked',
            'server': 'AmazonS3'
        },
        'RetryAttempts': 0
    },
    'IsTruncated': False,
    'KeyMarker': '',
    'VersionIdMarker': '',
    'Versions': [
        {
            'ETag': '"1b267619c4812cc46ee281747884ca50"',
            'Size': 2,
            'StorageClass': 'STANDARD',
            'Key': 'projects/learn-s3-versioning/business-report.txt',
            'VersionId': '36dbatEjGuecZop4MARBZMwfORr4k9Hc',
            'IsLatest': True,
            'LastModified': datetime.datetime(2023, 4, 15, 21, 35, 38, tzinfo=tzutc()),
            'Owner': {
                'DisplayName': 'sanhehu+awshsh-app-dev',
                'ID': 'd8468a5d68b2a6e29f0436ec9b64ed212e2b3a272d15429fbfc58a3c35ff4bf7'
            }
        },
        {
            'ETag': '"6654c734ccab8f440ff0825eb443dc7f"',
            'Size': 2,
            'StorageClass': 'STANDARD',
            'Key': 'projects/learn-s3-versioning/business-report.txt',
            'VersionId': 'BLoyX9X9xLmHSx8gLgNzWUKI7kwwkuNz',
            'IsLatest': False,
            'LastModified': datetime.datetime(2023, 4, 15, 21, 35, 37, tzinfo=tzutc()),
            'Owner': {
                'DisplayName': 'sanhehu+awshsh-app-dev',
                'ID': 'd8468a5d68b2a6e29f0436ec9b64ed212e2b3a272d15429fbfc58a3c35ff4bf7'
            }
        }
    ],
    'Name': '807388292768-us-east-1-data-versioned',
    'Prefix': 'projects/learn-s3-versioning/business-report.txt',
    'MaxKeys': 1000,
    'EncodingType': 'url'
}

Number of versions = 2


Next, we would like to test the deletion behavior. First, we put 3 more versions, so we have more versions to test with.

In [77]:
res = bsm.s3_client.put_object(Bucket=bucket, Key=key, Body="v3")
v3 = res["VersionId"]

res = bsm.s3_client.put_object(Bucket=bucket, Key=key, Body="v4")
v4 = res["VersionId"]

res = bsm.s3_client.put_object(Bucket=bucket, Key=key, Body="v5")
v5 = res["VersionId"]

print(f"v3 = {v3}")
print(f"v4 = {v4}")
print(f"v5 = {v5}")

v3 = FL9Bue.4UR_FC0h5el10gAzSzFlCQKPm
v4 = zufkTCbIA7N7DE0SPXS6AySBZzvRV5Oj
v5 = karfGCJYSeVX_d7q1fPW25syyIxKtXOl


Now we try to delete the object. What happens is that the latest object version is marked as deleted, but the content and the historical versions are still there.

In [78]:
res = bsm.s3_client.delete_object(Bucket=bucket, Key=key)
rprint(res)

# deleted_version = res["VersionId"]
# print(deleted_version, v5)
# assert deleted_version == v5
# print(f"deleted version = {deleted_version}")

{
    'ResponseMetadata': {
        'RequestId': '3P058SCF9ESE258R',
        'HostId': 'P4TZNbMrRnWGOuCcLRNfTg8G/cQMLNSD7n7q7R8hsgLlPhaspXLyd07ZSNfJXfT6wZD10MYDABfu0SsRJNN3aw==',
        'HTTPStatusCode': 204,
        'HTTPHeaders': {
            'x-amz-id-2': 
'P4TZNbMrRnWGOuCcLRNfTg8G/cQMLNSD7n7q7R8hsgLlPhaspXLyd07ZSNfJXfT6wZD10MYDABfu0SsRJNN3aw==',
            'x-amz-request-id': '3P058SCF9ESE258R',
            'date': 'Sat, 15 Apr 2023 21:36:08 GMT',
            'x-amz-version-id': '89GSZktJ_IIdNZJFe5carBbzupMlxgSE',
            'x-amz-delete-marker': 'true',
            'server': 'AmazonS3'
        },
        'RetryAttempts': 0
    },
    'DeleteMarker': True,
    'VersionId': '89GSZktJ_IIdNZJFe5carBbzupMlxgSE'
}

Then, let's try to get the object. S3 will get the latest version of this object, and find out it is marked as deleted, so it will return a 404 error.

In [79]:
res = bsm.s3_client.get_object(Bucket=bucket, Key=key)
# content = res["Body"].read().decode("utf-8")
# v = res["VersionId"]
#
# assert content == "v4"
# assert v == v4
#
# print(f"Content = {content}")
# print(f"Version Id = {v}")

NoSuchKey: An error occurred (NoSuchKey) when calling the GetObject operation: The specified key does not exist.

In [80]:
res = bsm.s3_client.get_object(Bucket=bucket, Key=key, VersionId=v4)
rprint(res)

{
    'ResponseMetadata': {
        'RequestId': '9VPZY36D5TVX8DF6',
        'HostId': 'U53YbOgyAM+oW9yxxOQl56tY0cAPnK0174esyBx1m/0+nqRMMxJgzA+AJ3rMd/l+EsvBJo9Ztuk=',
        'HTTPStatusCode': 200,
        'HTTPHeaders': {
            'x-amz-id-2': 'U53YbOgyAM+oW9yxxOQl56tY0cAPnK0174esyBx1m/0+nqRMMxJgzA+AJ3rMd/l+EsvBJo9Ztuk=',
            'x-amz-request-id': '9VPZY36D5TVX8DF6',
            'date': 'Sat, 15 Apr 2023 21:39:16 GMT',
            'last-modified': 'Sat, 15 Apr 2023 21:35:39 GMT',
            'etag': '"5ed3a3ff8e5c29979502b9030e8518e0"',
            'x-amz-server-side-encryption': 'AES256',
            'x-amz-version-id': 'zufkTCbIA7N7DE0SPXS6AySBZzvRV5Oj',
            'accept-ranges': 'bytes',
            'content-type': 'binary/octet-stream',
            'server': 'AmazonS3',
            'content-length': '2'
        },
        'RetryAttempts': 0
    },
    'AcceptRanges': 'bytes',
    'LastModified': datetime.datetime(2023, 4, 15, 21, 35, 39, tzinfo=tzutc()),
    'ContentLength': 2,
    'ETag': '"5ed3a3ff8e5c29979502b9030e8518e0"',
    'VersionId': 'zufkTCbIA7N7DE0SPXS6AySBZzvRV5Oj',
    'ContentType': 'binary/octet-stream',
    'ServerSideEncryption': 'AES256',
    'Metadata': {},
    'Body': <botocore.response.StreamingBody object at 0x1080abca0>
}

Now we can list all version again

In [59]:
res1 = bsm.s3_client.list_object_versions(Bucket=bucket, Prefix=key)
rprint(res1)

{
    'ResponseMetadata': {
        'RequestId': '9E6489A8KCB77S8T',
        'HostId': 'TfIuLv+2YBlabS5Pzp6mmCgQIQ3Ylds9BOzb5O+trF095DJYFeCafWdTwEeZ0KL6K5QvlRIbBF8=',
        'HTTPStatusCode': 200,
        'HTTPHeaders': {
            'x-amz-id-2': 'TfIuLv+2YBlabS5Pzp6mmCgQIQ3Ylds9BOzb5O+trF095DJYFeCafWdTwEeZ0KL6K5QvlRIbBF8=',
            'x-amz-request-id': '9E6489A8KCB77S8T',
            'date': 'Sat, 15 Apr 2023 21:30:23 GMT',
            'content-type': 'application/xml',
            'transfer-encoding': 'chunked',
            'server': 'AmazonS3'
        },
        'RetryAttempts': 0
    },
    'IsTruncated': False,
    'KeyMarker': '',
    'VersionIdMarker': '',
    'Versions': [
        {
            'ETag': '"4b6df01946f4919a3f5af8c1f0f9c3c5"',
            'Size': 2,
            'StorageClass': 'STANDARD',
            'Key': 'projects/learn-s3-versioning/business-report.txt',
            'VersionId': 'nAPU_RhaanKQt7UaDQfz02dqvyjKFj8I',
            'IsLatest': False,
            'LastModified': datetime.datetime(2023, 4, 15, 21, 27, tzinfo=tzutc()),
            'Owner': {
                'DisplayName': 'sanhehu+awshsh-app-dev',
                'ID': 'd8468a5d68b2a6e29f0436ec9b64ed212e2b3a272d15429fbfc58a3c35ff4bf7'
            }
        },
        {
            'ETag': '"5ed3a3ff8e5c29979502b9030e8518e0"',
            'Size': 2,
            'StorageClass': 'STANDARD',
            'Key': 'projects/learn-s3-versioning/business-report.txt',
            'VersionId': 'Kb0aorSWLcS_3N2knqz_9zwufm6F8xEC',
            'IsLatest': False,
            'LastModified': datetime.datetime(2023, 4, 15, 21, 27, tzinfo=tzutc()),
            'Owner': {
                'DisplayName': 'sanhehu+awshsh-app-dev',
                'ID': 'd8468a5d68b2a6e29f0436ec9b64ed212e2b3a272d15429fbfc58a3c35ff4bf7'
            }
        },
        {
            'ETag': '"43a03299a3c3fed3d8ce7b820f3aca81"',
            'Size': 2,
            'StorageClass': 'STANDARD',
            'Key': 'projects/learn-s3-versioning/business-report.txt',
            'VersionId': '8h9fqea4xGhze6eugeHgNbT5Jccozr7y',
            'IsLatest': False,
            'LastModified': datetime.datetime(2023, 4, 15, 21, 27, tzinfo=tzutc()),
            'Owner': {
                'DisplayName': 'sanhehu+awshsh-app-dev',
                'ID': 'd8468a5d68b2a6e29f0436ec9b64ed212e2b3a272d15429fbfc58a3c35ff4bf7'
            }
        },
        {
            'ETag': '"1b267619c4812cc46ee281747884ca50"',
            'Size': 2,
            'StorageClass': 'STANDARD',
            'Key': 'projects/learn-s3-versioning/business-report.txt',
            'VersionId': 'pKCvdIbB47hE3sAFZjDCqt.xqheRjkDD',
            'IsLatest': False,
            'LastModified': datetime.datetime(2023, 4, 15, 21, 26, 55, tzinfo=tzutc()),
            'Owner': {
                'DisplayName': 'sanhehu+awshsh-app-dev',
                'ID': 'd8468a5d68b2a6e29f0436ec9b64ed212e2b3a272d15429fbfc58a3c35ff4bf7'
            }
        },
        {
            'ETag': '"6654c734ccab8f440ff0825eb443dc7f"',
            'Size': 2,
            'StorageClass': 'STANDARD',
            'Key': 'projects/learn-s3-versioning/business-report.txt',
            'VersionId': 'dgxxL.IhdRvXQ85eTPi.Py82eu_kEN57',
            'IsLatest': False,
            'LastModified': datetime.datetime(2023, 4, 15, 21, 26, 53, tzinfo=tzutc()),
            'Owner': {
                'DisplayName': 'sanhehu+awshsh-app-dev',
                'ID': 'd8468a5d68b2a6e29f0436ec9b64ed212e2b3a272d15429fbfc58a3c35ff4bf7'
            }
        }
    ],
    'DeleteMarkers': [
        {
            'Owner': {
                'DisplayName': 'sanhehu+awshsh-app-dev',
                'ID': 'd8468a5d68b2a6e29f0436ec9b64ed212e2b3a272d15429fbfc58a3c35ff4bf7'
            },
            'Key': 'projects/learn-s3-versioning/business-report.txt',
            'VersionId': 'nSerOG9XapELdzYL9FAYflgBOyhHRM6f',
            'IsLatest': True,
            'LastModified': datetime.datetime(2023, 

In [24]:
_ = bsm.s3_client.put_object(
    Bucket=bucket,
    Key=f"{prefix}/folder/file1.txt",
    Body="file1-v1",
)
_ = bsm.s3_client.put_object(
    Bucket=bucket,
    Key=f"{prefix}/folder/file1.txt",
    Body="file1-v2",
)
_ = bsm.s3_client.put_object(
    Bucket=bucket,
    Key=f"{prefix}/folder/file2.txt",
    Body="file2-v1",
)
_ = bsm.s3_client.put_object(
    Bucket=bucket,
    Key=f"{prefix}/folder/file2.txt",
    Body="file2-v2",
)

In [25]:
res = bsm.s3_client.list_object_versions(
    Bucket=bucket,
    Prefix=f"{prefix}/folder/",
)
rprint(res)

{
    'ResponseMetadata': {
        'RequestId': 'NKKHZYCPWWYZ1J32',
        'HostId': 'EcVK6lDcEj3hyaERLo0bD+U8h9qK7x1sL3a8Xub0zre2n737mn6yHg2xLbWer2aTSme3XtCRse0=',
        'HTTPStatusCode': 200,
        'HTTPHeaders': {
            'x-amz-id-2': 'EcVK6lDcEj3hyaERLo0bD+U8h9qK7x1sL3a8Xub0zre2n737mn6yHg2xLbWer2aTSme3XtCRse0=',
            'x-amz-request-id': 'NKKHZYCPWWYZ1J32',
            'date': 'Sat, 15 Apr 2023 21:10:37 GMT',
            'content-type': 'application/xml',
            'transfer-encoding': 'chunked',
            'server': 'AmazonS3'
        },
        'RetryAttempts': 0
    },
    'IsTruncated': False,
    'KeyMarker': '',
    'VersionIdMarker': '',
    'Versions': [
        {
            'ETag': '"9131f6503b12d73dcfb9f641f517da04"',
            'Size': 8,
            'StorageClass': 'STANDARD',
            'Key': 'projects/learn-s3-versioning/folder/file1.txt',
            'VersionId': '9yWglLuLZXdWT7AT1KllqojQuJhScFIE',
            'IsLatest': True,
            'LastModified': datetime.datetime(2023, 4, 15, 21, 10, 29, tzinfo=tzutc()),
            'Owner': {
                'DisplayName': 'sanhehu+awshsh-app-dev',
                'ID': 'd8468a5d68b2a6e29f0436ec9b64ed212e2b3a272d15429fbfc58a3c35ff4bf7'
            }
        },
        {
            'ETag': '"112e3e27b3f77487cbed73cf1deacb5f"',
            'Size': 8,
            'StorageClass': 'STANDARD',
            'Key': 'projects/learn-s3-versioning/folder/file1.txt',
            'VersionId': 'tV4U5RRJMDLEwfKxQAVe_r74KzeUdWLC',
            'IsLatest': False,
            'LastModified': datetime.datetime(2023, 4, 15, 21, 10, 29, tzinfo=tzutc()),
            'Owner': {
                'DisplayName': 'sanhehu+awshsh-app-dev',
                'ID': 'd8468a5d68b2a6e29f0436ec9b64ed212e2b3a272d15429fbfc58a3c35ff4bf7'
            }
        },
        {
            'ETag': '"9131f6503b12d73dcfb9f641f517da04"',
            'Size': 8,
            'StorageClass': 'STANDARD',
            'Key': 'projects/learn-s3-versioning/folder/file1.txt',
            'VersionId': 'H6_lF33evUYLZsIoxd24kl7hoH0xXbuU',
            'IsLatest': False,
            'LastModified': datetime.datetime(2023, 4, 15, 21, 10, 14, tzinfo=tzutc()),
            'Owner': {
                'DisplayName': 'sanhehu+awshsh-app-dev',
                'ID': 'd8468a5d68b2a6e29f0436ec9b64ed212e2b3a272d15429fbfc58a3c35ff4bf7'
            }
        },
        {
            'ETag': '"112e3e27b3f77487cbed73cf1deacb5f"',
            'Size': 8,
            'StorageClass': 'STANDARD',
            'Key': 'projects/learn-s3-versioning/folder/file1.txt',
            'VersionId': '8AUw3qB1poFyPlhqHPN55jdHWqzrzHtT',
            'IsLatest': False,
            'LastModified': datetime.datetime(2023, 4, 15, 21, 10, 14, tzinfo=tzutc()),
            'Owner': {
                'DisplayName': 'sanhehu+awshsh-app-dev',
                'ID': 'd8468a5d68b2a6e29f0436ec9b64ed212e2b3a272d15429fbfc58a3c35ff4bf7'
            }
        },
        {
            'ETag': '"112e3e27b3f77487cbed73cf1deacb5f"',
            'Size': 8,
            'StorageClass': 'STANDARD',
            'Key': 'projects/learn-s3-versioning/folder/file1.txt',
            'VersionId': 'wboEjS_ceV0C4s.K4Twx1hDPkhGgBGXj',
            'IsLatest': False,
            'LastModified': datetime.datetime(2023, 4, 15, 21, 9, 54, tzinfo=tzutc()),
            'Owner': {
                'DisplayName': 'sanhehu+awshsh-app-dev',
                'ID': 'd8468a5d68b2a6e29f0436ec9b64ed212e2b3a272d15429fbfc58a3c35ff4bf7'
            }
        },
        {
            'ETag': '"20a8caaee5116a21cfcdbf2fbd5bc602"',
            'Size': 8,
            'StorageClass': 'STANDARD',
            'Key': 'projects/learn-s3-versioning/folder/file2.txt',
            'VersionId': 'IghSoYgJwy_JiYnsnx1.PA0WJSCA4ilQ',
            'IsLatest': True,
            'LastModified': datetime.datetime(2023, 4, 15, 21, 10, 29, tzinfo=tzutc()),
            'Owner': {
                'DisplayName': 'sanhehu+a

In [26]:
res = bsm.s3_client.list_object_versions(
    Bucket=bucket,
    Prefix=f"{prefix}/folder/file1.txt",
)
rprint(res)

{
    'ResponseMetadata': {
        'RequestId': '3J15A6YHFE3ZMJPD',
        'HostId': 'GSnmcn1hQxDKLpdmDG9pX4RFYFxV6P+D8JOaBm3m8lsltRLLwkLtWLLCRju3iSVLsyMuFSdJ/fs=',
        'HTTPStatusCode': 200,
        'HTTPHeaders': {
            'x-amz-id-2': 'GSnmcn1hQxDKLpdmDG9pX4RFYFxV6P+D8JOaBm3m8lsltRLLwkLtWLLCRju3iSVLsyMuFSdJ/fs=',
            'x-amz-request-id': '3J15A6YHFE3ZMJPD',
            'date': 'Sat, 15 Apr 2023 21:10:46 GMT',
            'content-type': 'application/xml',
            'transfer-encoding': 'chunked',
            'server': 'AmazonS3'
        },
        'RetryAttempts': 0
    },
    'IsTruncated': False,
    'KeyMarker': '',
    'VersionIdMarker': '',
    'Versions': [
        {
            'ETag': '"9131f6503b12d73dcfb9f641f517da04"',
            'Size': 8,
            'StorageClass': 'STANDARD',
            'Key': 'projects/learn-s3-versioning/folder/file1.txt',
            'VersionId': '9yWglLuLZXdWT7AT1KllqojQuJhScFIE',
            'IsLatest': True,
            'LastModified': datetime.datetime(2023, 4, 15, 21, 10, 29, tzinfo=tzutc()),
            'Owner': {
                'DisplayName': 'sanhehu+awshsh-app-dev',
                'ID': 'd8468a5d68b2a6e29f0436ec9b64ed212e2b3a272d15429fbfc58a3c35ff4bf7'
            }
        },
        {
            'ETag': '"112e3e27b3f77487cbed73cf1deacb5f"',
            'Size': 8,
            'StorageClass': 'STANDARD',
            'Key': 'projects/learn-s3-versioning/folder/file1.txt',
            'VersionId': 'tV4U5RRJMDLEwfKxQAVe_r74KzeUdWLC',
            'IsLatest': False,
            'LastModified': datetime.datetime(2023, 4, 15, 21, 10, 29, tzinfo=tzutc()),
            'Owner': {
                'DisplayName': 'sanhehu+awshsh-app-dev',
                'ID': 'd8468a5d68b2a6e29f0436ec9b64ed212e2b3a272d15429fbfc58a3c35ff4bf7'
            }
        },
        {
            'ETag': '"9131f6503b12d73dcfb9f641f517da04"',
            'Size': 8,
            'StorageClass': 'STANDARD',
            'Key': 'projects/learn-s3-versioning/folder/file1.txt',
            'VersionId': 'H6_lF33evUYLZsIoxd24kl7hoH0xXbuU',
            'IsLatest': False,
            'LastModified': datetime.datetime(2023, 4, 15, 21, 10, 14, tzinfo=tzutc()),
            'Owner': {
                'DisplayName': 'sanhehu+awshsh-app-dev',
                'ID': 'd8468a5d68b2a6e29f0436ec9b64ed212e2b3a272d15429fbfc58a3c35ff4bf7'
            }
        },
        {
            'ETag': '"112e3e27b3f77487cbed73cf1deacb5f"',
            'Size': 8,
            'StorageClass': 'STANDARD',
            'Key': 'projects/learn-s3-versioning/folder/file1.txt',
            'VersionId': '8AUw3qB1poFyPlhqHPN55jdHWqzrzHtT',
            'IsLatest': False,
            'LastModified': datetime.datetime(2023, 4, 15, 21, 10, 14, tzinfo=tzutc()),
            'Owner': {
                'DisplayName': 'sanhehu+awshsh-app-dev',
                'ID': 'd8468a5d68b2a6e29f0436ec9b64ed212e2b3a272d15429fbfc58a3c35ff4bf7'
            }
        },
        {
            'ETag': '"112e3e27b3f77487cbed73cf1deacb5f"',
            'Size': 8,
            'StorageClass': 'STANDARD',
            'Key': 'projects/learn-s3-versioning/folder/file1.txt',
            'VersionId': 'wboEjS_ceV0C4s.K4Twx1hDPkhGgBGXj',
            'IsLatest': False,
            'LastModified': datetime.datetime(2023, 4, 15, 21, 9, 54, tzinfo=tzutc()),
            'Owner': {
                'DisplayName': 'sanhehu+awshsh-app-dev',
                'ID': 'd8468a5d68b2a6e29f0436ec9b64ed212e2b3a272d15429fbfc58a3c35ff4bf7'
            }
        }
    ],
    'Name': '807388292768-us-east-1-data-versioned',
    'Prefix': 'projects/learn-s3-versioning/folder/file1.txt',
    'MaxKeys': 1000,
    'EncodingType': 'url'
}